In [ ]:
# -*- coding: utf-8 -*-
"""
vanilla_TF-IDF_vN.ipynb

This notebook is based on:
- https://medium.com/@coldstart_coder/understanding-and-implementing-tf-idf-in-python-a325d1301484
- https://www.kaggle.com/code/tylerpoff/understanding-and-implementing-tf-idf-in-python/notebook


instructions:
1. set query string
2. set corpus list of strings
3. create TF-IDF vector-matrix (pick an inverse_document_frequency variant)
4. search/sort for top-N results: tfidf_vector_search_top_n()
5. print results etc.
"""

# Query

In [1]:
query = """
machine learning algorithms
model building
scikit-learn, TensorFlow, and PyTorch.
models
predictive analytics
natural language processing
Agile/Scrum
sprints
stand-ups
Jira
Confluence
Waterfall
Kanban
"""


# Corpus (documents / text)

In [2]:
corpus = [
    "the Expert",
    "Proficient in Python programming and data analysis, with experience using libraries such as pandas, NumPy, and matplotlib for data manipulation and visualization. Able to write clean, efficient code and perform complex data analysis tasks.",
    "Experienced in working with SQL databases, including designing schemas, writing queries, and optimizing database performance. Familiarity with database management systems such as MySQL, PostgreSQL, and Oracle.",
    "Excellent communication skills, both written and verbal, with the ability to effectively collaborate with team members and stakeholders. Strong interpersonal skills and the ability to work well in a fast-paced, dynamic environment.",
    "Expert in machine learning algorithms and model building, with experience using tools such as scikit-learn, TensorFlow, and PyTorch. Able to design and implement machine learning models for a variety of applications, including predictive analytics and natural language processing.",
    "Experience with Agile/Scrum methodologies, including working in sprints, participating in daily stand-ups, and using tools such as Jira and Confluence. Familiarity with other project management methodologies, such as Waterfall and Kanban.",
    "Proficient in front-end web development, with experience using HTML, CSS, and JavaScript to build responsive, user-friendly websites. Familiarity with front-end frameworks such as React and Angular.",
    "Strong technical writing skills, with the ability to write clear, concise documentation that is easy for non-technical stakeholders to understand. Experience creating user manuals, technical specifications, and other technical documentation.",
    "Experienced in using data visualization tools such as Tableau and PowerBI to create interactive dashboards and reports. Able to effectively communicate complex data insights through clear, visually appealing visualizations.",
    "Experience with cloud platforms such as AWS, Google Cloud, and Azure, including deploying applications, managing infrastructure, and optimizing costs. Familiarity with cloud security best practices and compliance requirements.",
    "Strong problem-solving skills and attention to detail, with the ability to troubleshoot technical issues and identify root causes. Able to work independently and take ownership of projects from start to finish."
]


# Vanilla TF-IDF Code

In [7]:
# -*- coding: utf-8 -*-
"""
vanilla_TF-IDF_v8.ipynb

This notebook is based on:
- https://medium.com/@coldstart_coder/understanding-and-implementing-tf-idf-in-python-a325d1301484
- https://www.kaggle.com/code/tylerpoff/understanding-and-implementing-tf-idf-in-python/notebook


instructions:
1. set query string
2. set corpus list of strings
3. create TF-IDF vector-matrix (pick an inverse_document_frequency variant)
4. search/sort for top-N results: tfidf_vector_search_top_n()
5. print results etc.

"""

import math
import time


"""# query"""
query = query

corpus_unsplit = corpus



def term_frequency(word, document):
    return document.count(word) / len(document)
tf = term_frequency

# # non-plus-1 variant ("unsafe" variant)
# def inverse_document_frequency_unsafe(word, corpus):
#     count_of_documents = len(corpus)
#     count_of_documents_with_word = sum([1 for doc in corpus if word in doc])
#     idf = math.log10(count_of_documents/count_of_documents_with_word)
#     return idf

# sklearn variant
def inverse_document_frequency(word, corpus):
    count_of_documents = len(corpus) + 1
    count_of_documents_with_word = sum([1 for doc in corpus if word in doc]) + 1
    idf = math.log10(count_of_documents/count_of_documents_with_word) + 1
    return idf

idf = inverse_document_frequency

def TF_IDF(word, document, corpus):
    return tf(word, document) * idf(word, corpus)


"""# corpus of documents"""
split_corpus = [c.split() for c in corpus_unsplit]
num_documents = len(split_corpus)


"""### Optional Sample Target Word Analysis"""

"""
optional
"""
# target_word = "Expert"

# print("searching for the word '%s'"%target_word)
# for i, document in enumerate(split_corpus):
#     tf_score = tf(target_word, document)
#     idf_score = idf(target_word, split_corpus)
#     tf_idf_score = TF_IDF(target_word, document, split_corpus)

#     print("document %s: '%s'\n    tf score: %s\n    idf score: %s\n    tf_idf score:%s"%(i, document, tf_score, idf_score, tf_idf_score))
#     print("-"*30)

"""## word to vector mappings"""

"""
 create the word to vector mappings,
 we want each word to map to a unique point in our word vectors.
combine the complete corpus into a single list of words; remove duplicates.
use position in this list as the index for a word vector
"""
word_set = list(set(sum(split_corpus, [])))
# create a lookup for each word to it's index,
word_to_index = {word:i for i, word in enumerate(word_set)}

num_words = len(word_set)


"""## create the word vectors"""
# create an empty list to store our word vectors
word_vectors = []
for document in split_corpus:
    # for our new document create a new word vector
    new_word_vector = [0 for i in range(num_words)]

    # now we loop through each word in our document and compute the tf-idf score and populate our vector with it,
    # we only care about words in this document because words outside of it will remain zero
    for word in document:
        # get the score
        tf_idf_score = TF_IDF(word, document, split_corpus)
        # next get the index for this word in our word vector
        word_index = word_to_index[word]
        # populate the vector
        new_word_vector[word_index] = tf_idf_score

    # don't forget to add this new word vector to our list of existing word_vectors
    word_vectors.append(new_word_vector)


"""## one word vector in comparision to document"""
# # inspection
# print(corpus_unsplit[0])
# print(word_vectors[0])


"""## <b>Searching with TF-IDF Sparse Vectors</b>"""
query_keywords = query.split()

# now we loop through each documents word vector, get the tf-idf score for each keyword, sum them up and that is our tf-idf for that document,
# we keep track of the best document and return that as our result,
tf_idf_scores = []
best_document_index = 0
best_tf_idf = 0

for i, word_vector in enumerate(word_vectors):
    document_tf_idf_score_for_query = 0
    for word in query_keywords:
        # first do a check, does this word appear in our split_corpus of documents?
        # if not skip this keyword
        if word not in word_set:
            continue

        # get the index for this keyword and directly pull it from the word vector
        word_index = word_to_index[word]
        document_tf_idf_score_for_query += word_vector[word_index]
    tf_idf_scores.append(document_tf_idf_score_for_query) # keep track of all tf_idf scores, just in case we want to review them,

    """
    optional:
    top N list...TODO
    """
    # does this tf_idf score for this document beat our previous best?
    if document_tf_idf_score_for_query > best_tf_idf:
        best_tf_idf = document_tf_idf_score_for_query
        best_document_index = i


"""
Inspection & Study
"""
# from pprint import pprint
# # then print out our results
# # print("results of query: ", query)
# print("best tf_idf score sum for query: ", best_tf_idf)
# print("best document: ", corpus_unsplit[best_document_index])
# print("complete list of tf_idf scores: ", tf_idf_scores)
# from pprint import pprint
# print("tf_idf_scores -> ")
# pprint(tf_idf_scores)
##  pprint(corpus_unsplit)



def tfidf_vector_search_top_n(query, corpus, n):
    query_keywords = query.split()

    tf_idf_scores = []
    for i, word_vector in enumerate(word_vectors):
        document_tf_idf_score_for_query = 0
        for word in query_keywords:
            if word not in word_set:
                continue

            word_index = word_to_index[word]
            document_tf_idf_score_for_query += word_vector[word_index]
        tf_idf_scores.append((document_tf_idf_score_for_query, i))

    # Sort the TF-IDF scores in descending order
    tf_idf_scores.sort(reverse=True)

    # Extract the document indices from the top-N results
    top_n_document_indices = [index for _, index in tf_idf_scores[:n]]

    # Return the top-N documents and their TF-IDF scores
    top_n_documents = [corpus[index] for index in top_n_document_indices]
    top_n_tf_idf_scores = [score for score, _ in tf_idf_scores[:n]]

    return top_n_documents, top_n_tf_idf_scores


# Use TF-IDF

In [8]:
# Set This
how_many_results = 5

# Search
top_n = how_many_results
start_time = time.monotonic()  # timer
top_n_documents, top_n_tf_idf_scores = tfidf_vector_search_top_n(query, corpus_unsplit, top_n)
end_time = time.monotonic()  # timer
elapsed_time = end_time - start_time  # timer

print(f"Top-{top_n} results for query: {query}")
for i, (document, score) in enumerate(zip(top_n_documents, top_n_tf_idf_scores)):
    print(f"Result {i+1}:")
    print(f"TF-IDF score: {score}")
    print(f"Document: {document}\n")
# timer
print(f"Elapsed time: {elapsed_time} seconds")

Top-5 results for query: 
machine learning algorithms
model building
scikit-learn, TensorFlow, and PyTorch.
models
predictive analytics
natural language processing
Agile/Scrum
sprints
stand-ups
Jira
Confluence
Waterfall
Kanban

Result 1:
TF-IDF score: 0.7643492565507526
Document: Expert in machine learning algorithms and model building, with experience using tools such as scikit-learn, TensorFlow, and PyTorch. Able to design and implement machine learning models for a variety of applications, including predictive analytics and natural language processing.

Result 2:
TF-IDF score: 0.2725103043167461
Document: Experience with Agile/Scrum methodologies, including working in sprints, participating in daily stand-ups, and using tools such as Jira and Confluence. Familiarity with other project management methodologies, such as Waterfall and Kanban.

Result 3:
TF-IDF score: 0.12209277186934114
Document: Proficient in Python programming and data analysis, with experience using libraries such a